In [153]:
#Importing library 

import pandas as pd
import numpy as np
import imageio
import imgaug as ia
from PIL import Image
from imgaug import augmenters as iaa
import glob
import json
import os
import shutil
import sys
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
import keras.utils as image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
from copy import deepcopy
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from PIL import Image  
import numpy as np
from tqdm import tqdm
import tensorflow.keras as K
from sklearn.model_selection import train_test_split

In [9]:
#Importing library tensorflow to operate on VGG16 model

import tensorflow as tf
import tensorflow_datasets as tfds


import keras, os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

from tensorflow.python.util import deprecation
from tensorflow.keras import backend
from tensorflow.keras.applications.resnet50 import preprocess_input


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from tensorflow.keras.models import load_model

In [2]:
pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Mounting Google drive 

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
###Preparing dictionary for image labels 

label_map = {0:'Normal', 
             1:'Atelectasis',
             2:'Calcification', 
             3:'Consolidation', 
             4:'Effusion', 
             5:'Emphysema', 
             6:'Fibrosis', 
             7:'Fracture', 
             8:'Mass', 
             9: 'Nodule', 
             10:'Pneumothorax'}

rev_label_map = {v: k for k, v in label_map.items()}

In [7]:
###Mapping the train dataset labels with index and classes

##For mapping, we use json file of labels that were a part of our dataset

#In these steps, we first created a dataframe of train labels from json file and then created an array of those labels called y_train


with open('/content/drive/MyDrive/train.json') as user_file:
    file_contents = user_file.read()
parsed_json = json.loads(file_contents)

class_list = []
file_list = []
for i in parsed_json:
    class_list.append(list(set(i['syms'])))
    file_list.append(i['file_name'])

for i in class_list:
    if len(i)==0:
        i.append(label_map[0])


df = pd.DataFrame(data=class_list)
df['label'] = pd.Series(class_list)
df = df['label']
df_process = pd.get_dummies(df.apply(pd.Series).stack()).sum(level=0)
df_process['file_name'] = pd.Series(file_list)
df_clean=df_process.drop('file_name',axis = 1)
y_train = np.array(df_clean)
len(y_train)

<ipython-input-7-7fc2b8502d92>:26: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df_process = pd.get_dummies(df.apply(pd.Series).stack()).sum(level=0)


3001

In [8]:
#Depiction of how the array of labels look like 

y_train[:10] 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [5]:
###Mapping the test dataset labels with index and classes

##For mapping, we use json file of labels that were a part of our dataset

#In these steps, we first created a dataframe of test labels from json file and then created an array of those labels called y_test 

with open('/content/drive/MyDrive/test.json') as user_file:
    file_contents = user_file.read()
parsed_test_json = json.loads(file_contents)

test_class_list = []
test_file_list = []
for i in parsed_test_json:
    test_class_list.append(list(set(i['syms'])))
    test_file_list.append(i['file_name'])


for i in test_class_list:
    if len(i)==0:
        i.append(label_map[0])

test_df = pd.DataFrame(data=test_class_list)
test_df['label'] = pd.Series(test_class_list)
test_df = test_df['label']
test_df_process = pd.get_dummies(test_df.apply(pd.Series).stack()).sum(level=0)
test_df_process['file_name'] = pd.Series(test_file_list)
test_df_clean=test_df_process.drop('file_name',axis = 1)
y_test = np.array(test_df_clean)
len(y_test)

<ipython-input-5-e35d70279e09>:25: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  test_df_process = pd.get_dummies(test_df.apply(pd.Series).stack()).sum(level=0)


542

In [6]:
#Depiction of how the array of labels look like 

y_test[:10] 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]], dtype=uint8)

In [10]:
#We converted the .jpg images of test and train data into a numpy array file for the ease of processing in computer vision models 

train_data = np.load('/content/drive/MyDrive/xray/train_image.npy', encoding='bytes')
test_data = np.load('/content/drive/MyDrive/xray/test_image.npy', encoding='bytes')

In [11]:
#Checking the shape of training data 

train_data.shape

(3001, 1024, 1024)

In [12]:
#Checking the shape of testing data 

test_data.shape

(542, 1024, 1024)

In [13]:
#Reshaping the training dataset to reduce their size from 1024 to 224 and from 3-dimensional to 4-dimensional 

train_new = np.zeros((3001, 300, 300, 1))
for i in range(3001):
  train_new[i] = np.reshape(cv2.resize(train_data[i], (300,300)), (300,300,1))

In [14]:
#Checking new shape of training dataset 

train_new.shape

(3001, 300, 300, 1)

In [15]:
#Reshaping the testing dataset to reduce their size from 1024 to 224 and from 3-dimensional to 4-dimensional 

test_new = np.zeros((542, 300, 300, 1))
for i in range(542):
  test_new[i] = np.reshape(cv2.resize(test_data[i], (300,300)), (300,300,1))

In [16]:
#Checking new shape of testing dataset 

test_new.shape

(542, 300, 300, 1)

In [17]:
#stratifying data while splitting train and validation data
from skmultilearn.model_selection import iterative_train_test_split
X_train, y_train, X_val, y_val = iterative_train_test_split(train_new, y_train, test_size = 0.1)

In [18]:
###Setting VGG16 model
##Added the last two additional model layers where channels 

#All the hidden layers use ReLU activation and the last Dense layer uses Sigmoid activation with 11 channels 

def vgg_16(weights = 'imagenet', include_top = True):

    model = Sequential()
    model.add(Conv2D(input_shape = (300, 300, 1), filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))

    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))

    model.add(Flatten())
    model.add(Dense(units = 4096, activation = "relu"))
    model.add(Dense(units = 4096, activation = "relu"))
    model.add(Dense(units = 11, activation = "sigmoid"))

    opt = Adam(lr = 0.1)
   
    return model
  
print(vgg_16().summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 300, 300, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 300, 300, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 150, 150, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 150, 150, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 150, 150, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 75, 75, 128)      0         
 2D)                                                    

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
#Setting up and compiling the model
model = vgg_16()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()])

In [21]:
# Normalizing and augmenting the data
aug = ImageDataGenerator(horizontal_flip=True, rotation_range=10, featurewise_center=True, featurewise_std_normalization=True,  fill_mode="nearest")
aug.fit(X_train)

In [24]:
# Training tuned VGG16 model by feeding label dataset through "aug" for fitting the model to get training and validation accuracy per epoch

batch_size=32
model.fit(x=aug.flow(X_train, y_train, batch_size=32),validation_data=(X_val, y_val), steps_per_epoch = len(X_train)//batch_size, epochs = 10)


Epoch 1/10
84/84 [==============================] - 4600s 55s/step - loss: 0.3961 - binary_accuracy: 0.8361 - auc: 0.7252 - val_loss: 0.3833 - val_binary_accuracy: 0.8380 - val_auc: 0.7574
Epoch 2/10
84/84 [==============================] - 4494s 54s/step - loss: 0.3660 - binary_accuracy: 0.8461 - auc: 0.7584 - val_loss: 0.3793 - val_binary_accuracy: 0.8380 - val_auc: 0.7623
Epoch 3/10
84/84 [==============================] - 4552s 54s/step - loss: 0.3660 - binary_accuracy: 0.8474 - auc: 0.7581 - val_loss: 0.3789 - val_binary_accuracy: 0.8380 - val_auc: 0.7603
Epoch 4/10
84/84 [==============================] - 4627s 55s/step - loss: 0.3650 - binary_accuracy: 0.8481 - auc: 0.7599 - val_loss: 0.3794 - val_binary_accuracy: 0.8380 - val_auc: 0.7608
Epoch 5/10
84/84 [==============================] - 4585s 55s/step - loss: 0.3646 - binary_accuracy: 0.8472 - auc: 0.7608 - val_loss: 0.3814 - val_binary_accuracy: 0.8380 - val_auc: 0.7604
Epoch 6/10
84/84 [==============================] - 454

In [32]:
#Fetching probability predictions of each validation and testing images 
val_pred = model.predict(X_val)
np.save('/content/drive/MyDrive/vgg_val_out_final.npy', val_pred)  

test_pred = model.predict(test_new)
np.save('/content/drive/MyDrive/vgg_test_out_final.npy', test_pred)  

17/17 [==============================] - 256s 15s/step


In [ ]:
#Model Ending